During testing how long time to check all empty file in a update folder, I found that there are may symobls appear
empty, but they actually in the production folder.
Further analysis shows that seems these are the symbols that dislisted from stock market before. So the data won't
update anymore. Somehow, it still showed up in the stock list.
I wonder if this is just by design or I should try another approach from start. Like request all data every time
and remove duplicates afterwards.


In [1]:
import os
import pandas as pd
import numpy as np
import Paths
import api_keys
import requests
import json
import time
import datetime
import TD_API
import time_utilities
import utilities

In [6]:
# Check if an element is in both list
listA = [1, 2, 3, 4, 5]
listB = [ 1,2,3,5, 6, 7, 8]
element_in_both_list = set(listA) & set(listB)
element_in_both_list
missing = set(listA) - set(listB)
missing

{4}

In [2]:
df = pd.DataFrame()
df.to_csv(Paths.Daily_Data_Tracker, index=False)

In [9]:
mycolumns = ['Date','New Symbol','Missing Symbol', 'Empty Symbol', 'New Empty Symbol']
df = pd.DataFrame(columns=mycolumns)
df.to_csv(Paths.Daily_Data_Tracker, index=False)

In [2]:
update_folders = utilities.list_subfolders(Paths.Daily_Data_Update_Folder)
update_folders.sort()
update_folders

['./Data/Daily/Updates/2021-11-28',
 './Data/Daily/Updates/2021-11-29',
 './Data/Daily/Updates/2021-11-30']

In [26]:
# create dafaframe from list of dict
df_A = pd.DataFrame(result)
df_A.sort_values(by='earilest_time', inplace=True)
df_A

,symbol,earilest_time
7002,SPGI.csv,21600000
10246,HWM.csv,108000000
519,NC.csv,108000000
258,MRO.csv,108000000
5710,CAG.csv,71128800000
...,...,...
8460,BRD.U.csv,1637733600000
10166,NYMTZ.csv,1637733600000
1490,OPP-B.csv,1637906400000
7725,JMAC.csv,1637906400000


In [29]:
df_A['earilest_time'].iloc[0]

21600000

In [34]:
time_utilities.epoch_to_eastern(df_A['earilest_time'].iloc[10])

'1984-11-01 01:00:00'

In [35]:
AAPL = pd.read_csv('./AAPL.csv')
AAPL

,Unnamed: 0,open,high,low,close,volume,datetime
0,0,0.130023,0.130023,0.124442,0.124442,175302400,473493600000
1,1,0.126674,0.130023,0.126674,0.126674,166611200,473580000000
2,2,0.126674,0.127232,0.125000,0.126674,137267200,473666400000
3,3,0.126674,0.127232,0.126116,0.126116,170912000,473925600000
4,4,0.126116,0.127232,0.125000,0.125000,141120000,474012000000
...,...,...,...,...,...,...,...
9300,9300,161.680000,165.700000,161.000000,161.020000,117467889,1637560800000
9301,9301,161.120000,161.800000,159.060100,161.410000,96041899,1637647200000
9302,9302,160.750000,162.140000,159.640000,161.940000,69463623,1637733600000
9303,9303,159.565000,160.450000,156.360000,156.810000,76959752,1637906400000


In [37]:
time_utilities.epoch_to_eastern(AAPL['datetime'].iloc[0])

'1985-01-02 01:00:00'

In [38]:
AAPL_PRO = pd.read_csv(Paths.Daily_Data_Production_Folder + '/AAPL.csv')

In [39]:
AAPL_PRO

,open,high,low,close,volume,timestamp,datetime
0,159.565000,160.450000,156.360000,156.810000,76959752,1637906400000,2021-11-26 01:00:00
1,160.750000,162.140000,159.640000,161.940000,69463623,1637733600000,2021-11-24 01:00:00
2,161.120000,161.800000,159.060100,161.410000,96041899,1637647200000,2021-11-23 01:00:00
3,161.680000,165.700000,161.000000,161.020000,117467889,1637560800000,2021-11-22 01:00:00
4,157.650000,161.020000,156.532800,160.550000,117305597,1637301600000,2021-11-19 01:00:00
...,...,...,...,...,...,...,...
9299,0.126116,0.127232,0.125000,0.125000,141120000,474012000000,1985-01-08 01:00:00
9300,0.126674,0.127232,0.126116,0.126116,170912000,473925600000,1985-01-07 01:00:00
9301,0.126674,0.127232,0.125000,0.126674,137267200,473666400000,1985-01-04 01:00:00
9302,0.126674,0.130023,0.126674,0.126674,166611200,473580000000,1985-01-03 01:00:00


In [42]:
list_files = [f for f in os.listdir(Paths.Daily_Data_Production_Folder) if f.endswith('.csv')]


In [43]:
list_files

['RIV.csv',
 'ANTE.csv',
 'BRZE.csv',
 'CSCO.csv',
 'PRI.csv',
 'INDIW.csv',
 'STL-A.csv',
 'ARGU.csv',
 'NZF.csv',
 'TYRA.csv',
 'AIRC.csv',
 'HLNE.csv',
 'DSCF.csv',
 'SPXB.csv',
 'GXIIU.csv',
 'COLIU.csv',
 'HUBS.csv',
 'GPL.csv',
 'AFTR.U.csv',
 'SBEA.csv',
 'VCAR.csv',
 'SBGI.csv',
 'HTLFP.csv',
 'UFCS.csv',
 'TEAF.csv',
 'RJZ.csv',
 'QDEU.csv',
 'C-P.CL.csv',
 'GFED.csv',
 'DRH-A.csv',
 'AIRT.csv',
 'SITC.csv',
 'TLIS.csv',
 'TLGA+.csv',
 'NXN.csv',
 'SVSVW.csv',
 'BNY.csv',
 'SPXU.csv',
 'NUVB+.csv',
 'FRC-C.CL.csv',
 'PPA.csv',
 'RHE.csv',
 'SBEV.csv',
 'MDCP.csv',
 'LILM.csv',
 'TGVCW.csv',
 'ISSC.csv',
 'KNDI.csv',
 'EFT.csv',
 'USTB.csv',
 'TRNS.csv',
 'XSHD.csv',
 'BBSA.csv',
 'RDW+.csv',
 'PTOCU.csv',
 'BCI.csv',
 'ACII.U.csv',
 'BBRE.csv',
 'NTZ.csv',
 'LBPH.csv',
 'LYFE.csv',
 'AEIS.csv',
 'UAL.csv',
 'TROW.csv',
 'LEVYW.csv',
 'ISRG.csv',
 'EGP.csv',
 'TRND.csv',
 'GDRX.csv',
 'AUBAP.csv',
 'QDIV.csv',
 'VPCBU.csv',
 'ISO.csv',
 'EFC.csv',
 'EEX.csv',
 'NVR.csv',
 'DDLS